<a href="https://colab.research.google.com/github/aim56009/Bias_GAN/blob/master/code/tas_prec_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports 

In [1]:
is_colab = False

if is_colab==False:
    colab_path = "bias_gan_t_p/"
else:
    colab_path = "/content/gdrive/MyDrive/bias_gan_t_p"

In [2]:
detrend_data = False

In [3]:
"""
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
"""

"\ngpu_info = !nvidia-smi\ngpu_info = '\n'.join(gpu_info)\nif gpu_info.find('failed') >= 0:\n  print('Not connected to a GPU')\nelse:\n  print(gpu_info)\n"

In [4]:
if is_colab == True:
    from google.colab import drive
    drive.mount("/content/gdrive")

In [5]:
if is_colab == True:
    !git clone https://github.com/aim56009/bias_gan_t_p.git

In [6]:
%%capture
!pip install pytorch_lightning
from pytorch_lightning.loggers import TensorBoardLogger
!pip install basemap
!pip install importlib-metadata==4.0.1
!pip install xarray==0.18.1
!pip install torchvision
!pip install cftime

In [7]:
if is_colab == False:
    import os
    os.chdir('/dss/dsshome1/0D/ge74xuf2/')

In [8]:
import os
import xarray as xr
import torch
import json
import glob
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as pl
import argparse
import pathlib
import cv2
import matplotlib


from tensorboard.backend.event_processing import event_accumulator
from pytorch_lightning.callbacks import Callback
from datetime import datetime
from io import BytesIO
from PIL import Image
from dataclasses import dataclass, field
from typing import List


from bias_gan_t_p.code.src.model import CycleGAN, Generator, DataModule                     

from bias_gan_t_p.code.src.data import TestData, CycleDataset
from bias_gan_t_p.code.src.utils import get_version, set_environment, get_checkpoint_path, save_config, log_transform, inv_norm_transform, inv_log_transform, inv_norm_minus1_to_plus1_transform, norm_minus1_to_plus1_transform 
from bias_gan_t_p.code.src.plots import PlotAnalysis, plot_basemap
from bias_gan_t_p.code.src.callbacks import get_cycle_gan_callbacks, MAE_Callback
from bias_gan_t_p.code.src.inference import Inference, EvaluateCheckpoints, create_folder

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


# Data.py

In [9]:
#Config_adjusted_trafo = Config
#nbr_reconstruction_examples = 1
#Config_adjusted_trafo.transforms = Config_adjusted_trafo.transformations
#dataset = CycleDataset('train', Config_adjusted_trafo)

In [10]:
#datamodule = DataModule(Config, training_batch_size = Config.train_batch_size, test_batch_size = Config.test_batch_size)
#datamodule.setup("fit")
#train_loader = datamodule.train_dataloader()

#for batch_idx, data in enumerate(train_loader):
#    print(data["A"])
#    print(data["B"])

In [11]:
#track_lat_mean = True
#plt_hist=True
#accelerator= "gpu"

#train_cycle_gan(Config(),validation=False,track_lat_mean=track_lat_mean,plt_hist=plt_hist)

# Main training loop

## define MAE callback

In [12]:
class MAE_Callback(Callback):
    def __init__(self,logger,checkpoint_path,config, validation=True, lat_mean=False, plt_hist=False):
        self.MAE_list_pr = []
        self.MAE_list_t = []
        self.logger = logger
        self.checkpoint_path = checkpoint_path
        self.config = config
        self.version = get_version(config.date,config.time)
        self.validation = validation
        self.lat_mean = lat_mean
        self.plt_hist = plt_hist
        

    def on_train_epoch_end(self, trainer, pl_module):
        checkpoint_files = glob.glob(str(self.checkpoint_path) + '/*.ckpt')
        if not checkpoint_files:
            test_data_ = None
        else:
            last_checkpoint = max(checkpoint_files, key=os.path.getctime)
            data = EvaluateCheckpoints(checkpoint_path=last_checkpoint, config_path=self.config.config_path + self.version + "/config_model.json", save_model=True,validation=self.validation, version=self.version)
            _, reconstruction_data = data.run()
            test_data_ = data.get_test_data()


        if test_data_ is None or not test_data_:
            print("No test data available.")
            return

        gan_data = getattr(test_data_, 'gan')
        era5_data = getattr(test_data_, "era5")
        
        bias_pr = gan_data.precipitation.mean('time') - era5_data.precipitation.mean('time') 
        print("GAN-OBS precipitation",f" \t \t MAE: {abs(bias_pr).values.mean():2.3f} [mm/d]")
        self.MAE_list_pr.append(abs(bias_pr).values.mean())
        print("MAE_list precipitation:",self.MAE_list_pr)
        self.log('MAE p', abs(bias_pr).values.mean())
        print("")
        bias_t = gan_data.tas.mean('time') - era5_data.tas.mean('time') 
        print("GAN-OBS tas",f" \t \t MAE: {abs(bias_t).values.mean():2.3f} [K]")
        self.MAE_list_t.append(abs(bias_t).values.mean())
        print("MAE_list tas:",self.MAE_list_t)
        self.log('MAE t', abs(bias_t).values.mean())
        print("")

        if test_data_ is not None and self.lat_mean==True:
            data_era5_pr = era5_data.precipitation.mean(dim=("longitude", "time"))
            data_era5_t = era5_data.tas.mean(dim=("lon", "time"))
            
            data_gan_pr= gan_data.precipitation.mean(dim=("longitude", "time"))
            data_gan_t= gan_data.tas.mean(dim=("longitude", "time"))
            
            plt.figure()
            
            plt.plot(data_gan_pr.latitude, data_gan_pr.data,
                      label="gan precipitation",
                      alpha=0.9,
                      linestyle='-',
                      linewidth=2,
                      color="red")
            
            plt.plot(data_era5_pr.latitude, data_era5_pr.data,
                      label="era5 precipitation",
                      alpha=1,
                      linestyle='--',
                      linewidth=2,
                      color="black")
            
            #plt.ylim(0,3)
            plt.xlim(25,58)
            plt.xlabel('Latitude')
            plt.ylabel('Mean precipitation [mm/d]')
            plt.grid()
            plt.legend(loc='upper right')  
            
            buf = BytesIO()
            plt.savefig(buf, format='png')
            buf.seek(0)
            im = Image.open(buf)
            img = torchvision.transforms.ToTensor()(im)
            
            self.logger.experiment.add_image(f"latitudinal_mean precipitation", img, trainer.current_epoch)
            
            
            ###
            plt.figure()
            
            plt.plot(data_gan_t.latitude, data_gan_t.data,
                      label="gan temperature",
                      alpha=0.9,
                      linestyle='-',
                      linewidth=2,
                      color="red")
            
            plt.plot(data_era5_t.lat, data_era5_t.data,
                      label="era5 temperature",
                      alpha=1,
                      linestyle='--',
                      linewidth=2,
                      color="black")
            
            #plt.ylim(0,3)
            plt.xlim(25,58)
            plt.xlabel('Latitude')
            plt.ylabel('Mean temperature [K]')
            plt.grid()
            plt.legend(loc='upper right')
            #plt.show()
            ###
            
            buf = BytesIO()
            plt.savefig(buf, format='png')
            buf.seek(0)
            im = Image.open(buf)
            img = torchvision.transforms.ToTensor()(im)
            
            self.logger.experiment.add_image(f"latitudinal_mean temperature", img, trainer.current_epoch)

        if test_data_ is not None and self.plt_hist==True:
            data_gan_pr = getattr(test_data_, "gan").precipitation.values.flatten()
            data_era5_pr = getattr(test_data_, "era5").precipitation.values.flatten()
            plt.figure()
            _ = plt.hist(data_gan_pr,
                        bins=100,
                        histtype='step',
                        log=True,
                        label="gan",
                        alpha=0.9,
                        density=True,
                        linewidth=2,
                        color="red")
            
            _ = plt.hist(data_era5_pr,
                        bins=100,
                        histtype='step',
                        log=True,
                        label="era5",
                        alpha=1,
                        density=True,
                        linewidth=2,
                        color="black")

            plt.xlabel('Precipitation [mm/d]')
            plt.ylabel('Histogram')
            #plt.xlim(0,400)
            plt.grid()
            plt.legend(loc='upper right')

            #plt.show()
            buf = BytesIO()
            plt.savefig(buf, format='png')
            buf.seek(0)
            im_ = Image.open(buf)
            img_ = torchvision.transforms.ToTensor()(im_)
            
            self.logger.experiment.add_image(f"histogram precipitation", img_, trainer.current_epoch)
            
            
            data_gan_t = getattr(test_data_, "gan").tas.values.flatten()
            data_era5_t = getattr(test_data_, "era5").tas.values.flatten()
            plt.figure()
            _ = plt.hist(data_gan_t,
                        bins=100,
                        histtype='step',
                        log=True,
                        label="gan",
                        alpha=0.9,
                        density=True,
                        linewidth=2,
                        color="red")
            
            _ = plt.hist(data_era5_t,
                        bins=100,
                        histtype='step',
                        log=True,
                        label="era5",
                        alpha=1,
                        density=True,
                        linewidth=2,
                        color="black")

            plt.xlabel('Temperature [K]')
            plt.ylabel('Histogram')
            #plt.xlim(0,400)
            plt.grid()
            plt.legend(loc='upper right')

            #plt.show()
            buf = BytesIO()
            plt.savefig(buf, format='png')
            buf.seek(0)
            im_ = Image.open(buf)
            img_ = torchvision.transforms.ToTensor()(im_)
            
            self.logger.experiment.add_image(f"histogram", img_, trainer.current_epoch)

## Train Cycle GAN

In [13]:
def train_cycle_gan(config, pretrain_path=False,validation=True,track_lat_mean=False,plt_hist=False ):
    """ Main routing to train the Cycle GAN """

    config = Config()
    global version
    version = get_version(config.date,config.time)
    print(f'Running model: {version}')
    checkpoint_path = get_checkpoint_path(config, version)
    set_environment()

    tb_logger = TensorBoardLogger(config.tensorboard_path,name="",version=version,default_hp_metric=False)
    
    
    create_folder(f"{colab_path}results/{version}")

    save_config(config, version)
    
    mse_callback = MAE_Callback(tb_logger,checkpoint_path,config,validation,lat_mean=track_lat_mean,plt_hist=plt_hist)
    
    
    trainer = pl.Trainer(callbacks=[mse_callback] + get_cycle_gan_callbacks(checkpoint_path),
                         devices=1,
                         max_epochs = config.epochs,
                         precision = 16, 
                         num_sanity_val_steps = 1,
                         logger = tb_logger,
                         log_every_n_steps = config.log_every_n_steps,
                         deterministic = False,
                         accelerator=accelerator,
                         enable_model_summary=False) 
    

    datamodule = DataModule(config, training_batch_size = config.train_batch_size, test_batch_size = config.test_batch_size)
    datamodule.setup("fit")
    
    
    if pretrain_path==False:
        print("no pretraining")
        if config.epochs==1:
            model = CycleGAN(d_lr=config.d_lr, g_lr=config.g_lr, beta_1=config.beta_1, beta_2=config.beta_2,
                           epoch_decay = config.epochs,running_bias=config.running_bias,
                           num_resnet_blocks=config.num_resnet_layer, default_nbr_resnet=config.default_nbr_resnet)
        else:
            model = CycleGAN(d_lr=config.d_lr, g_lr=config.g_lr, beta_1=config.beta_1, beta_2=config.beta_2,
                           epoch_decay = config.epochs // 2,running_bias=config.running_bias,
                           num_resnet_blocks=config.num_resnet_layer, default_nbr_resnet=config.default_nbr_resnet)

    else:
        print("using pretrained model with path:",pretrain_path)
        model = CycleGAN(d_lr=config.d_lr, g_lr=config.g_lr, beta_1=config.beta_1, beta_2=config.beta_2,
                       epoch_decay = config.epochs // 2, running_bias=config.running_bias,
                       num_resnet_blocks=config.num_resnet_layer, 
                       default_nbr_resnet=config.default_nbr_resnet).load_from_checkpoint(pretrain_path)

    trainer.fit(model, datamodule)

    print('Training finished')
    return model

# Config

In [14]:
@dataclass
class Config:
    """ 
    Training configuration parameters. For model evaluation parameters see
    src/configuration.py.
    """
    
    scratch_path: str = f'{colab_path}results'
    tensorboard_path: str = f'{scratch_path}/'
    checkpoint_path: str = f'{scratch_path}/'
    config_path: str = f'{scratch_path}/'
    
    #if detrend_data==True:
    #    model_pr_path: str = f"{colab_path}data/detrend_pr_gfdl-esm4_historical_regionbox_1979-2014.nc"
    #    era5_pr_path: str = f"{colab_path}data/detrend_pr_W5E5v2.0_regionbox_era5_1979-2014.nc"
    #    model_t_path: str = f"{colab_path}data/detrend_tas_gfdl-esm4_historical_regionbox_1979-2014.nc"
    #    era5_t_path: str = f"{colab_path}data/detrend_tas_W5E5v2.0_regionbox_1979-2014.nc"
    
    if detrend_data==False:
        model_pr_path: str = f"{colab_path}data/pr_gfdl-esm4_historical_regionbox_1979-2014.nc"
        era5_pr_path: str = f"{colab_path}data/pr_W5E5v2.0_regionbox_era5_1979-2014.nc"
        model_t_path: str = f"{colab_path}data/tas_gfdl-esm4_historical_regionbox_1979-2014.nc"
        era5_t_path: str = f"{colab_path}data/tas_W5E5v2.0_regionbox_1979-2014.nc"
       

    results_path: str = f'{scratch_path}/'
    projection_path: str = None

    train_start: int = 1979
    train_end: int = 2000 #2000 
    valid_start: int = 2001 #was 2001
    valid_end: int = 2004
    test_start: int = 2004
    test_end: int = 2014
    
    model_name: str = 'tibet_gan'

    epochs: int = 2 # set to 250 for reproduction
    train_batch_size: int = 1
    test_batch_size: int = 64
    transforms: List = field(default_factory=lambda: ['log', 'normalize_minus1_to_plus1'])
    transformations = ['log', 'normalize_minus1_to_plus1']
    rescale: bool = False
    epsilon: float = 0.001 #0.0001
    lazy: bool = False
    log_every_n_steps: int = 10 ### was 10
    norm_output: bool = True
    running_bias: bool = False

    d_lr = 2e-4
    g_lr = 2e-4
    beta_1 = 0.5
    beta_2 = 0.999
    epoch_decay = 200
    

    time = datetime.now().time().strftime("%Hh_%Mm_%Ss")
    date = datetime.now().date().strftime("%Y_%m_%d")
    
    default_nbr_resnet=True
    num_resnet_layer=6


def main():
    _ = train_cycle_gan(Config())

#Run

In [15]:
print("training on detrended data:", detrend_data)

training on detrended data: False


In [16]:
do_training = False
from_skratch = True

track_lat_mean = True
plt_hist=True

runtime_instance = "2023_02_24_17h_27m_29s"

print("training on detrended data:", detrend_data)

if do_training == True:
    accelerator= "gpu"

    if from_skratch == True:
        train_cycle_gan(Config(),validation=False,track_lat_mean=track_lat_mean,plt_hist=plt_hist)
        

    if from_skratch == False:
        train_cycle_gan(Config(),f"{colab_path}results/{runtime_instance}/last.ckpt",validation=True,track_lat_mean=track_lat_mean,plt_hist=plt_hist)

training on detrended data: False


# Evaluation

## Run Evaluation


In [17]:
if do_training==False: 
  version_ = runtime_instance
else:
  version_ = version


checkpoint_path = f"{colab_path}results/{version_}/last.ckpt" 
config_path = f"{colab_path}results/{version_}/config_model.json"

data = EvaluateCheckpoints(checkpoint_path=checkpoint_path, config_path=config_path, save_model=True, version=version_)

In [18]:
test_data, reconstruct_data = data.run()
test_data = data.get_test_data()

Start inference:
MODE: TESTING
calling get_netcdf_result
saving model to path: 2023_02_24_17h_27m_29s/gan.nc

Test set period: 2004-01-01T00:00:00.000000000 - 2014-12-31T00:00:00.000000000

Mean precipitation [mm/d]:
ERA5: 1.017
Climate Model: 1.017
GAN:  0.883

Mean temperature [K]:
ERA5: 281.394
Climate Model: 281.420
GAN:  279.048
